In [1]:
from pyspark.ml.evaluation import RegressionEvaluator
from pyspark.ml.recommendation import ALS
from pyspark.sql import SparkSession
from pyspark.sql.functions import col

In [2]:
# Create a spark session
spark = SparkSession.builder.appName("CollaborativeFiltering-v2").getOrCreate()

23/01/26 23:12:52 WARN Utils: Your hostname, Hasti-2.local resolves to a loopback address: 127.0.0.1; using 192.168.1.4 instead (on interface en0)
23/01/26 23:12:52 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address


Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


23/01/26 23:12:53 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
23/01/26 23:12:53 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.
23/01/26 23:12:53 WARN Utils: Service 'SparkUI' could not bind on port 4041. Attempting port 4042.


In [3]:
# Read in the ratings dataset
ratings = spark.read.csv("./Dataset/ratings.csv", header=True, inferSchema=True)

In [4]:
# Split the dataset into training and testing sets
(training, test) = ratings.randomSplit([0.8, 0.2])

# Build the ALS model
als = ALS(maxIter=5, regParam=0.01, userCol="user_id", itemCol="game_id", ratingCol="rating")

In [5]:
# Fit the model to the training data
model = als.fit(training)

# Generate predictions on the test data
predictions = model.transform(test)

23/01/26 23:13:05 WARN InstanceBuilder$NativeBLAS: Failed to load implementation from:dev.ludovic.netlib.blas.JNIBLAS
23/01/26 23:13:05 WARN InstanceBuilder$NativeBLAS: Failed to load implementation from:dev.ludovic.netlib.blas.ForeignLinkerBLAS
23/01/26 23:13:06 WARN InstanceBuilder$NativeLAPACK: Failed to load implementation from:dev.ludovic.netlib.lapack.JNILAPACK


In [6]:
# Make recommendations for all users
userRecs = model.recommendForAllUsers(5)

In [7]:
# Show the recommendations for a specific user
userRecs.filter(col("user_id") == 1).show()

+-------+--------------------+
|user_id|     recommendations|
+-------+--------------------+
|      1|[{8605, 10.531054...|
+-------+--------------------+



In [8]:
# Read in the games dataset
games = spark.read.csv("./Dataset/games.csv", header=True, inferSchema=True)

In [9]:
# get game name based on game id
def getGameName(game_id):
    return games.filter(col("game_id") == game_id).collect()[0].name
# get 5 recommendations game name for specific user
def getRecsForUser(user_id):
    recommendations = userRecs.filter(col("user_id") == user_id).collect()[0].recommendations
    for recommendation in recommendations:
        print(getGameName(recommendation.game_id))


In [10]:
getRecsForUser(10)

Crash 'N' Burn
The Franz Kafka Videogame
Super Meat Boy Forever
Zendoku
CALENDULA


In [ ]:
# Stop the spark session
spark.stop()